In [1]:
import asyncio
import aiohttp
import warnings
import pandas as pd
import geopandas as geo
import numpy as np
from tqdm import tqdm
import os
import plotly.express as px
import uuid
import osmnx as ox
from keplergl import KeplerGl
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPolygon, MultiPoint
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent="myGeocoder")
import requests

ox.settings.log_console=True
warnings.simplefilter(action='ignore', category=FutureWarning)

#osmnx config timeout
timeout = 10000
ox.config(timeout=timeout)

/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/anaconda3/lib/python3.9/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


In [ ]:
async def fetch_data(session, geom, tags):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = ox.geometries.geometries_from_polygon_query(geom, tags)
    async with session.get(overpass_url, params={'data': overpass_query}) as response:
        return await response.text()

async def get_data(geom, tags):
    async with aiohttp.ClientSession() as session:
        responses = await asyncio.gather(*[fetch_data(session, geom, tags) for _ in range(5)])
    return responses

us = geo.read_file('geojson/us_boundary.geojson')
geom = us.geometry.iloc[0]
tags = {'power': True} 

loop = asyncio.get_event_loop()

try:
    future = asyncio.run_coroutine_threadsafe(get_data(geom, tags), loop)
    data = future.result()
    osm_pull = pd.concat([ox.geometries.geometries_from_xml(data[i]).reset_index() for i in range(len(data))])
    osm_pull.to_csv('us_pull.csv', index=False)
finally:
    loop.close()
